# Test download request store

In [1]:
%run ./setup-logging.ipynb
%run ./jupyter-test-support.ipynb
%run ./create_container.ipynb

Created [container]
  * app_root
  * blob_root_directory
  * blob_store
  * download_request_store
  * environment_name
  * event_publisher
  * mongodb_client
  * mongodb_credentials
  * mongodb_db_name
  * queue_receiver
  * queue_receiver_manager
  * queue_sender
  * rabbitmq_connection
  * rabbitmq_url
  * url_download_handler
  * url_downloaded_handler
  * url_info_store
  * url_store


In [2]:
download_request_store = container.download_request_store()

urls = [
    'https://www.towardsdatascience.com/',
    'https://www.towardsdatascience.com/article1',
    'https://www.newyorktimes.com/',
    'https://www.towardsdatascience.com/',
    'https://www.google.com/',
    'https://www.bing.com/',
    'https://www.foobar.com/'
]

In [3]:
%%cell_testsuite

@cell_test
def test_size(self):
    self.assertTrue(download_request_store.size() >= 0)

@cell_test
def test_clear(self):
    download_request_store.clear()
    self.assertEqual(0, download_request_store.size())
    self.assertEqual(0, len(download_request_store.list()))

..
----------------------------------------------------------------------
Ran 2 tests in 0.038s

OK


In [4]:
%%cell_testsuite

@cell_test
def test_add(self):
    download_request_store.clear()
    
    seen_urls = set()
    for url in urls:
        expected = url not in seen_urls
        r = download_request_store.add(url)
        self.assertEqual(expected, r)
        seen_urls.add(url)
        print(f"[{r}] -> {url}")
        
    # Adding again should return false
    for url in urls:
        r = download_request_store.add(url)
        self.assertFalse(r)
        
    # Check the list in db
    requests = download_request_store.list()
    
    urls_in_db = { r['url'] for r in requests }
    self.assertSetEqual(seen_urls, urls_in_db)

[True] -> https://www.towardsdatascience.com/
[True] -> https://www.towardsdatascience.com/article1
[True] -> https://www.newyorktimes.com/
[False] -> https://www.towardsdatascience.com/
[True] -> https://www.google.com/
[True] -> https://www.bing.com/
[True] -> https://www.foobar.com/
.
----------------------------------------------------------------------
Ran 1 test in 0.024s

OK


In [5]:
%%cell_testsuite

@cell_test
def test_remove(self):
    download_request_store.clear()
    
    for url in urls:
        download_request_store.add(url)
    self.assertTrue(download_request_store.size() > 0)
    
    for url in urls:
        download_request_store.remove(url)
        
    self.assertEqual(0, download_request_store.size())

.
----------------------------------------------------------------------
Ran 1 test in 0.025s

OK
